# CSCI 4253 / 5253 - Lab #4 - Patent Problem with Spark DataFrames
<div>
 <h2> CSCI 4283 / 5253 
  <IMG SRC="https://www.colorado.edu/cs/profiles/express/themes/cuspirit/logo.png" WIDTH=50 ALIGN="right"/> </h2>
</div>

### Solution using Dataframe: Shreyas Gopalakrishna

This [Spark cheatsheet](https://s3.amazonaws.com/assets.datacamp.com/blog_assets/PySpark_SQL_Cheat_Sheet_Python.pdf) is useful as is [this reference on doing joins in Spark dataframe](http://www.learnbymarketing.com/1100/pyspark-joins-by-example/).

The [DataBricks company has one of the better reference manuals for PySpark](https://docs.databricks.com/spark/latest/dataframes-datasets/index.html) -- they show you how to perform numerous common data operations such as joins, aggregation operations following `groupBy` and the like.

In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

The following aggregation functions may be useful -- [these can be used to aggregate results of `groupby` operations](https://docs.databricks.com/spark/latest/dataframes-datasets/introduction-to-dataframes-python.html#example-aggregations-using-agg-and-countdistinct). More documentation is at the [PySpark SQL Functions manual](https://spark.apache.org/docs/2.3.0/api/python/pyspark.sql.html#module-pyspark.sql.functions). Feel free to use other functions from that library.

In [2]:
from pyspark.sql.functions import col, count, countDistinct

Create our session as described in the tutorials

In [2]:
spark = SparkSession \
    .builder \
    .appName("Lab4-Dataframe") \
    .master("local[*]")\
    .getOrCreate()

Read in the citations and patents data and check that the data makes sense. Note that unlike in the RDD solution, the data is automatically inferred to be Integer() types.

In [3]:
citations = spark.read.load('cite75_99.txt.gz',
            format="csv", sep=",", header=True,
            compression="gzip",
            inferSchema="true")

In [4]:
citations.show(5)

+-------+-------+
| CITING|  CITED|
+-------+-------+
|3858241| 956203|
|3858241|1324234|
|3858241|3398406|
|3858241|3557384|
|3858241|3634889|
+-------+-------+
only showing top 5 rows



In [5]:
patents = spark.read.load('apat63_99.txt.gz',
            format="csv", sep=",", header=True,
            compression="gzip",
            inferSchema="true")

In [6]:
patents.count()

2923922

In [7]:
patents.show(5)

+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+
| PATENT|GYEAR|GDATE|APPYEAR|COUNTRY|POSTATE|ASSIGNEE|ASSCODE|CLAIMS|NCLASS|CAT|SUBCAT|CMADE|CRECEIVE|RATIOCIT|GENERAL|ORIGINAL|FWDAPLAG|BCKGTLAG|SELFCTUB|SELFCTLB|SECDUPBD|SECDLWBD|
+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+
|3070801| 1963| 1096|   null|     BE|   null|    null|      1|  null|   269|  6|    69| null|       1|    null|    0.0|    null|    null|    null|    null|    null|    null|    null|
|3070802| 1963| 1096|   null|     US|     TX|    null|      1|  null|     2|  6|    63| null|       0|    null|   null|    null|    null|    null|    null|    null|    null|    null|
|3070803| 1963| 1096|   null|     US|     IL|    null|      1|  null|     2|  6|    6

#### Joining Citations data with Patent data such that the cited number matches the patent info

In [8]:
innerJoin = citations.join(patents, citations.CITED == patents.PATENT)

In [9]:
innerJoin.show(10)

+-------+-------+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+
| CITING|  CITED| PATENT|GYEAR|GDATE|APPYEAR|COUNTRY|POSTATE|ASSIGNEE|ASSCODE|CLAIMS|NCLASS|CAT|SUBCAT|CMADE|CRECEIVE|RATIOCIT|GENERAL|ORIGINAL|FWDAPLAG|BCKGTLAG|SELFCTUB|SELFCTLB|SECDUPBD|SECDLWBD|
+-------+-------+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+
|3951073|3071083|3071083| 1963| 1096|   null|     AT|   null|    null|      1|  null|   104|  5|    55| null|       3|    null|    0.0|    null|    null|    null|    null|    null|    null|    null|
|4401034|3071083|3071083| 1963| 1096|   null|     AT|   null|    null|      1|  null|   104|  5|    55| null|       3|    null|    0.0|    null|    null|    null|    null|    null|    null|    null|
|4802

#### Selecting only the Cited, Cited State and Citing info from the join table. Renaming the table name so that it can be resued without errors

In [10]:
mr1 = innerJoin.select("CITED","POSTATE","CITING")
mr1 = mr1.toDF("CITED1", "POSTATE1", "CITING1")

In [11]:
mr1.show(10)

+-------+--------+-------+
| CITED1|POSTATE1|CITING1|
+-------+--------+-------+
|3071083|    null|3951073|
|3071083|    null|4401034|
|3071083|    null|4802416|
|3071452|    null|4045189|
|3071452|    null|4192656|
|3071753|      MN|3949375|
|3071753|      MN|4120573|
|3071753|      MN|4271479|
|3071753|      MN|4345315|
|3071753|      MN|4496943|
+-------+--------+-------+
only showing top 10 rows



#### Joining the generated table again with the patent data to obtain the state of citing 

In [12]:
inter = mr1.join(patents, mr1.CITING1 == patents.PATENT)

In [13]:
inter.show(10)

+-------+--------+-------+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+
| CITED1|POSTATE1|CITING1| PATENT|GYEAR|GDATE|APPYEAR|COUNTRY|POSTATE|ASSIGNEE|ASSCODE|CLAIMS|NCLASS|CAT|SUBCAT|CMADE|CRECEIVE|RATIOCIT|GENERAL|ORIGINAL|FWDAPLAG|BCKGTLAG|SELFCTUB|SELFCTLB|SECDUPBD|SECDLWBD|
+-------+--------+-------+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+
|3638586|      CA|3858527|3858527| 1975| 5485|   1973|     CH|   null|  336690|      3|    10|   108|  6|    69|    7|       1|  0.4286|    0.0|     0.0|     2.0| 19.5714|     0.0|     0.0|     0.0|     0.0|
|3699902|      OH|3858527|3858527| 1975| 5485|   1973|     CH|   null|  336690|      3|    10|   108|  6|    69|    7|       1|  0.4286|    0.0|     0.0|     2.0| 19.57

#### Selecting only the Cited, Cited State, Citing and Citing state to obtain the intermediate table.

In [14]:
mr2 = inter.select("CITED1","POSTATE1","CITING1","POSTATE")
mr2 = mr2.toDF("CITED2", "POSTATE2", "CITING2","POSTATE2_CT")

In [15]:
mr2.show(10)

+-------+--------+-------+-----------+
| CITED2|POSTATE2|CITING2|POSTATE2_CT|
+-------+--------+-------+-----------+
|3638586|      CA|3858527|       null|
|3699902|      OH|3858527|       null|
|3602157|      TX|3858527|       null|
|3815160|      NY|3858597|         MT|
|3675252|      AZ|3858597|         MT|
|3741706|      OH|3859029|         NY|
|3685936|      OH|3859029|         NY|
|3368197|      MI|3859627|         MI|
|3226674|    null|3859627|         MI|
|3359539|      CA|3859627|         MI|
+-------+--------+-------+-----------+
only showing top 10 rows



#### Fildering based on same state, grouping on citing and counting the number of same state citations. Sorting to get largest at the top.

In [16]:
a = mr2.filter("POSTATE2 = POSTATE2_CT").groupBy("CITING2").count().sort("count", ascending=False)

In [17]:
a.show(10)

+-------+-----+
|CITING2|count|
+-------+-----+
|5959466|  125|
|5983822|  103|
|6008204|  100|
|5952345|   98|
|5998655|   96|
|5958954|   96|
|5936426|   94|
|5978329|   90|
|5951547|   90|
|5739256|   90|
+-------+-----+
only showing top 10 rows



In [18]:
type(a)

pyspark.sql.dataframe.DataFrame

#### Verifing count based on patent id individially 

In [19]:
a.filter(a['CITING2'] == 5887243 ).show()
#a.filter($"CITING2" = 6009554).collect()

+-------+-----+
|CITING2|count|
+-------+-----+
|5887243|   72|
+-------+-----+



#### Joining the obtained table back to patent info to obtain the same state citations in each patent info

In [20]:
finalJoin = patents.join(a, patents.PATENT == a.CITING2, "left_outer").na.fill(0, "count")
finalJoin.show(10)

+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+-------+-----+
| PATENT|GYEAR|GDATE|APPYEAR|COUNTRY|POSTATE|ASSIGNEE|ASSCODE|CLAIMS|NCLASS|CAT|SUBCAT|CMADE|CRECEIVE|RATIOCIT|GENERAL|ORIGINAL|FWDAPLAG|BCKGTLAG|SELFCTUB|SELFCTLB|SECDUPBD|SECDLWBD|CITING2|count|
+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+-------+-----+
|3070853| 1963| 1096|   null|     US|     FL|    null|      1|  null|    49|  5|    59| null|       0|    null|   null|    null|    null|    null|    null|    null|    null|    null|   null|    0|
|3071083| 1963| 1096|   null|     AT|   null|    null|      1|  null|   104|  5|    55| null|       3|    null|    0.0|    null|    null|    null|    null|    null|    null|    null|   null|    0|
|3071452| 1963|

#### Final output selecting only the required columns

In [21]:
output = finalJoin.select("PATENT","GYEAR","GDATE","APPYEAR","COUNTRY","POSTATE","ASSIGNEE","ASSCODE","CLAIMS","NCLASS","CAT","SUBCAT","CMADE","CRECEIVE","RATIOCIT","GENERAL","ORIGINAL","FWDAPLAG","BCKGTLAG","SELFCTUB","SELFCTLB","SECDUPBD","SECDLWBD","count")

In [22]:
output.show(10)

+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+-----+
| PATENT|GYEAR|GDATE|APPYEAR|COUNTRY|POSTATE|ASSIGNEE|ASSCODE|CLAIMS|NCLASS|CAT|SUBCAT|CMADE|CRECEIVE|RATIOCIT|GENERAL|ORIGINAL|FWDAPLAG|BCKGTLAG|SELFCTUB|SELFCTLB|SECDUPBD|SECDLWBD|count|
+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+-----+
|3070853| 1963| 1096|   null|     US|     FL|    null|      1|  null|    49|  5|    59| null|       0|    null|   null|    null|    null|    null|    null|    null|    null|    null|    0|
|3071083| 1963| 1096|   null|     AT|   null|    null|      1|  null|   104|  5|    55| null|       3|    null|    0.0|    null|    null|    null|    null|    null|    null|    null|    0|
|3071452| 1963| 1096|   null|     BE|   null|    null| 

In [23]:
output.count()

2923922

In [24]:
output.sort("count", ascending=False).show(20)

+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+-----+
| PATENT|GYEAR|GDATE|APPYEAR|COUNTRY|POSTATE|ASSIGNEE|ASSCODE|CLAIMS|NCLASS|CAT|SUBCAT|CMADE|CRECEIVE|RATIOCIT|GENERAL|ORIGINAL|FWDAPLAG|BCKGTLAG|SELFCTUB|SELFCTLB|SECDUPBD|SECDLWBD|count|
+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+-----+
|5959466| 1999|14515|   1997|     US|     CA|    5310|      2|  null|   326|  4|    46|  159|       0|     1.0|   null|  0.6186|    null|  4.8868|  0.0455|   0.044|    null|    null|  125|
|5983822| 1999|14564|   1998|     US|     TX|  569900|      2|  null|   114|  5|    55|  200|       0|   0.995|   null|  0.7201|    null|   12.45|     0.0|     0.0|    null|    null|  103|
|6008204| 1999|14606|   1998|     US|     CA|  749584| 